In [ ]:
import chromadb
import pandas as pd

In [ ]:
chroma_client = chromadb.Client()

En caso que se desee persistir la base de datos

In [ ]:
# from chromadb.config import Settings
# chroma_client = chromadb.Client(
#     Settings(
#         chroma_db_impl="duckdb+parquet",
#         persist_directory='my_personal_vector_db',
#     )
# )

Nombramos la colección que vamos a utilizar como "my_news"

In [ ]:
collection_name = "my_news"

**chroma_client.list_collections()** retorna una lista con información acerca de las colecciones vigentes dentro de la base persistida

En caso que la colección ya exista, y que exista una con el mismo nombre que aquella que intentamos crea, va a ser eliminada para comenzar el proceso nuevamente partiendo desde el inicio

In [ ]:
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
    chroma_client.delete_collection(name=collection_name)
else:
    print(f"Creating collection: '{collection_name}'")
    collection = chroma_client.create_collection(name=collection_name)


Para embeddings customizados, es necesario crear una nueva **función de embedding** que permita procesar texto

```python
    collection = chroma_client.create_collection(name="my_collection", embedding_function=emb_fn)
```


In [ ]:
pdf = pd.read_csv("labelled_newscatcher_coloured.csv", index_col=0)

In [ ]:
pdf_subset = pdf.head(1000)

In [ ]:
pdf_subset

## Operaciones CRUD
Sobre esta base de datos vamos a utilizar operaciones CRUD (Create, Read, Update & Delete) con sintaxis similar a MongoDB

Al momento de insertar documentos sobre una base de datos de vectores, se insertan los documentos que van a ser vectorizados, en conjunto con los IDs que van a ser usados para identificar dichos documentos y la metadata asociada a los mismos

Así como pueden insertarse documentos, Chromadb soporta la inserción de **embeddings** de forma directa sin necesidad de especificar algún documento, esto resulta útil para realizar búsquedas con texto sobre bases de datos de imágenes con modelos como [CLIP](https://huggingface.co/openai/clip-vit-large-patch14) capaces de manejar ambos tipos de información (texto y visual)

In [ ]:
collection.add(
    documents=pdf_subset["title"][:100].to_list(),
    metadatas=[{"topic": topic} for topic in pdf_subset["topic"][:100].tolist()],
    ids=[f"id{x}" for x in range(100)],
)

In [ ]:
import json

results = collection.query(
    query_texts=["space"],
    # query_texts=["espacio"],
    n_results=10
)

print(json.dumps(results, indent=4))

In [ ]:
import json

results = collection.query(
    query_texts=["bombs"],
    n_results=3
)

print(json.dumps(results, indent=4))

In [ ]:
collection.query(
    query_texts=["space"],
    # en el caso de operaciones de filtrado usando  "where", 
    # pueden darse operadores $and, $or, $ge, etc, de la misma forma que se dieron con MongoDB
    where={"topic": "SCIENCE"}, 
    n_results=10,
)

Borramos el primer elemento de la colección

In [ ]:
collection.delete(
    ids=["id0"],
)

Verificamos que ya no se encuentra disponible

In [ ]:
collection.get(
    ids=["id0"],
)

Ahora realizamos un ejemplo de actualización de un documento

In [ ]:
collection.get(
    ids=["id2"],
)

Para el documento 2, vamos a cambiar su tópico de "SCIENCE" a "TECHNOLOGY"

In [ ]:
collection.update(
    ids=["id2"],
    metadatas={"topic": "TECHNOLOGY"}
)

Verificamos que el mismo haya cambiado

In [ ]:
collection.get(
    ids=["id2"],
)

Ahora se va a armar un pipeline de Q&A sencillo utilizando como fuente de datos, la base de datos que acabamos de crear con ChromaDB.

El hito es poder proveer de contexto a algún modelo generativo de lenguaje (en este caso en particular a GPT2) tratando de eficientizar el performance del modelo y tratar de acortar la ventana de contexto necesaria para que el modelo pueda funcionar correctamente

Recordar que GPT2 es una versión gratuita y un modelo antiguo de GPT4, no es de esperar que cuente con el mismo performance que modelos como GPT3 en adelante

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    # cache_dir='cache'
)

lm_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # cache_dir='cache',
)

pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    device_map="auto",
)

In [ ]:
results

In [ ]:
results = collection.query(
    query_texts=["space"],
    # query_texts=["espacio"],
    n_results=10
)

print(json.dumps(results, indent=4))

In [ ]:
question = "What's the latest news on space development?"

Establecemos el contexto necesario para responder la pregunta del usuario

In [ ]:
context = " ".join([f"\n#{str(i)}" for i in results["documents"][0]])
print(context)

In [ ]:
prompt_template = f"Relevant context: {context}\n\n The user's question: {question}"
print(prompt_template)

In [ ]:
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])

Para más información sobre como incorporar GPT4 y armar un chatbot de Q&A, visitar [Embeddings con OPEN AI](https://docs.trychroma.com/embeddings) para comprender cómo puede integrarse ChromaDB 